In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from google.colab import drive
drive.mount('/content/drive')

# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(['Arousal_Value', 'Valence_Value', 'frameTime'], axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)

# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation results
mse_a = []
mae_a = []
rmse_a = []
r2_a = []
f1_a = []
accuracy_a = []
precision_a = []
recall_a = []

# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X_reshaped):
    X_train, X_val = X_reshaped[train_index], X_reshaped[test_index]
    y_a_train, y_a_val = y_a[train_index], y_a[test_index]

    # Split the validation set further into val and test sets
    val_size = len(X_val) // 2
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_a_val, y_a_test = y_a_val[:val_size], y_a_val[val_size:]

    # Build the BiLSTM model for Arousal
    model_a = Sequential()
    model_a.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model_a.add(Flatten())

    # Add DNN layers with dropout
    dnn_layers = [512, 512, 512]  # Modify this list as needed
   # dropout_rate = 0.3  # Modify the dropout rate as needed

    for layer_size in dnn_layers:
        model_a.add(Dense(layer_size, activation='relu'))
        #model_a.add(Dropout(dropout_rate))

    model_a.add(Dense(1))
    model_a.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

    # Train the Arousal model
    model_a.fit(X_train, y_a_train, validation_data=(X_val, y_a_val), epochs=25, batch_size=32)

    # Evaluate the model on the test set
    predictions_a = model_a.predict(X_test)

    # Reshape predictions_a to match the shape of y_a_test
    predictions_a = predictions_a.reshape(y_a_test.shape)

    # Calculate additional evaluation metrics
    mse_a.append(mean_squared_error(y_a_test, predictions_a))
    mae_a.append(mean_absolute_error(y_a_test, predictions_a))
    rmse_a.append(np.sqrt(mean_squared_error(y_a_test, predictions_a)))
    r2_a.append(r2_score(y_a_test, predictions_a))

    # Convert regression predictions to binary labels
    threshold = 0
    binary_predictions_a = (predictions_a >= threshold).astype(int)
    y_a_test_binary = (y_a_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_a.append(f1_score(y_a_test_binary, binary_predictions_a))
    accuracy_a.append(accuracy_score(y_a_test_binary, binary_predictions_a))
    precision_a.append(precision_score(y_a_test_binary, binary_predictions_a))
    recall_a.append(recall_score(y_a_test_binary, binary_predictions_a))

# Calculate average performance across all folds
average_mse_a = np.mean(mse_a)
average_mae_a = np.mean(mae_a)
average_rmse_a = np.mean(rmse_a)
average_r2_a = np.mean(r2_a)
average_f1_a = np.mean(f1_a)
average_accuracy_a = np.mean(accuracy_a)
average_precision_a = np.mean(precision_a)
average_recall_a = np.mean(recall_a)

print(f'Average Arousal MSE: {average_mse_a:.4f}')
print(f'Average Arousal MAE: {average_mae_a:.4f}')
print(f'Average Arousal RMSE: {average_rmse_a:.4f}')
print(f'Average Arousal R2 Score: {average_r2_a:.4f}')
print(f'Average Arousal F1 Score: {average_f1_a:.4f}')
print(f'Average Arousal Accuracy: {average_accuracy_a:.4f}')
print(f'Average Arousal Precision: {average_precision_a:.4f}')
print(f'Average Arousal Recall: {average_recall_a:.4f}')

Mounted at /content/drive
Epoch 1/25
2654/2654 [==============================] - 68s 22ms/step - loss: 0.0478 - val_loss: 0.0556
Epoch 2/25
2654/2654 [==============================] - 57s 21ms/step - loss: 0.0405 - val_loss: 0.0494
Epoch 3/25
2654/2654 [==============================] - 58s 22ms/step - loss: 0.0379 - val_loss: 0.0452
Epoch 4/25
2654/2654 [==============================] - 58s 22ms/step - loss: 0.0365 - val_loss: 0.0445
Epoch 5/25
2654/2654 [==============================] - 60s 23ms/step - loss: 0.0354 - val_loss: 0.0421
Epoch 6/25
2654/2654 [==============================] - 59s 22ms/step - loss: 0.0349 - val_loss: 0.0403
Epoch 7/25
2654/2654 [==============================] - 58s 22ms/step - loss: 0.0343 - val_loss: 0.0416
Epoch 8/25
2654/2654 [==============================] - 60s 23ms/step - loss: 0.0336 - val_loss: 0.0395
Epoch 9/25
2654/2654 [==============================] - 58s 22ms/step - loss: 0.0332 - val_loss: 0.0406
Epoch 10/25
2654/2654 [===============